# CS295/395: Secure Distributed Computation
## Homework 9

In [2]:
import numpy as np
import random
import hashlib

## The Petersen Graph

[The Petersen Graph](https://en.wikipedia.org/wiki/Petersen_graph) has 10 vertices and 15 edges, and can be colored with 3 colors.

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Petersen_graph_3-coloring.svg/220px-Petersen_graph_3-coloring.svg.png "Title")

In [3]:
coloring = {
    # outer five nodes, clockwise from top
    0: 'red',
    1: 'blue', 
    2: 'green',
    3: 'red',
    4: 'blue',
    # inner five nodes, clockwise from top
    5: 'blue',
    6: 'red',
    7: 'red',
    8: 'green',
    9: 'green'
}
print('Number of nodes:', len(coloring))

Number of nodes: 10


In [4]:
edges = [
    # outer shape, clockwise from top
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 0),
    # inner shape, clockwise from top
    (5, 0), (5, 7),
    (6, 1), (6, 8),
    (7, 2), (7, 9),
    (8, 3), (8, 5),
    (9, 4), (9, 6)
]
print('Number of edges:', len(edges))

Number of edges: 15


## Background

**Your goal in this assignment** is to implement a zero-knowledge interactive proof protocol that allows the Prover to convince the Verifier that the Prover knows a valid 3-coloring for the Petersen graph - without revealing the coloring.

Reference the description in lecture on 10/30 and 11/02, and [Matt Green's blog post](https://blog.cryptographyengineering.com/2014/11/27/zero-knowledge-proofs-illustrated-primer/).

## Protocol

The Prover and Verifier perform the following steps $n^2$ times, where $n$ is the number of vertices in the graph:

- **Step 1: shuffle and commit.** The Prover randomizes the colors and commits to the coloring. The Prover sends the commitment to the Verifier.
- **Step 2: challenge.** The Verifier picks a random edge in the graph.
- **Step 3: response.** The Prover opens the commitment for the two vertices connected by the chosen edge. If the two colors are the same, the Verifier rejects.

## Question 1 (40 points)

Implement the above protocol for an interactive zero-knowledge proof of the graph coloring.

In [8]:
class Prover:
    def shuffle_and_commit(self):
        colors = ['red', 'blue', 'green']
        shuffled_colors = colors.copy()
        random.shuffle(shuffled_colors)

        color_map = dict(zip(colors, shuffled_colors))

        self.shuffled_coloring = {}
        for node, color in coloring.items():
            self.shuffled_coloring[node] = color_map[color]

        self.committed_coloring_with_noise = {}
        committed_coloring = {}
        for node, color in self.shuffled_coloring.items():
            # create some noise so that the verifier can't tell if two nodes have the same color
            noise = ":" + str(random.random())

            # commit to the color and noise
            commitment = hashlib.sha256((color + noise).encode()).hexdigest()

            self.committed_coloring_with_noise[node] = color, noise
            committed_coloring[node] = commitment
            
        return (committed_coloring)
    
    def response(self, edge):
        # get the color + noise of each node in the edge
        color1, noise1 = self.committed_coloring_with_noise[edge[0]]
        color2, noise2 = self.committed_coloring_with_noise[edge[1]]
        
        c1 = (color1, noise1)
        c2 = (color2, noise2)
        
        # ask the verifier to check the response
        return (c1, c2)

    def create_example(self):
        commitment = self.shuffle_and_commit()
        random_oracle = int(hashlib.sha256(str(commitment).encode()).hexdigest(), 16)
        challenge_edge = edges[random_oracle % len(edges)]
        return (commitment, challenge_edge, self.response(challenge_edge))


class Verifier:

    def check(self, commitment, challenge_edge, response):
        c1, c2 = response

        color1, noise1 = c1
        color2, noise2 = c2

        # check that the challenge edge was picked fairly
        random_oracle = int(hashlib.sha256(str(commitment).encode()).hexdigest(), 16)
        real_challenge_edge = edges[random_oracle % len(edges)]

        assert challenge_edge == real_challenge_edge

        # check that the colors are different
        assert color1 != color2

        # check that the commitments are correct
        assert hashlib.sha256((color1 + noise1).encode()).hexdigest() == commitment[challenge_edge[0]]
        assert hashlib.sha256((color2 + noise2).encode()).hexdigest() == commitment[challenge_edge[1]]
        

def run_protocol():
    P = Prover()
    V = Verifier()
    examples = [P.create_example() for _ in range(15**2)]
    for ex in examples:
        V.check(*ex)
    
run_protocol()

## Question 2 (10 points)

In 2-5 sentences, argue that the protocol is zero-knowledge (it doesn't reveal anything about the witness).

With every round, we are recoloring the graph with a shuffled color mapping. This means that no information can be corelated round to round and we are not able to build up intuition about a coloring as the rounds go on. The verifier gets to pick any two adjacent nodes (so picking an edge) and then we "lift up the hats" by revealing the color and noise we committed to. We must include the noise in this otherwise the hashes themselves reveal the colorings. Until we supply the color and noise for verification, the verifier cannot extact any info from the commitments alone. After providing the color and noise, the verifier can confirm that the two nodes they picked are, in fact, different colors, and that we had committed to those colors before we knew the random nodes they would pick.

## Question 3 (10 points)

What is the probability that a cheating Prover is *not* caught in this protocol, and why?

Assuming there is only a single inconsitency in the graph coloring, there is only one edge that verifier can challenge that will reveal the cheating, and that inconsitency can easily be "shifted" around the graph to a random place at the commitment stage.

Let $E$ = number of edges

Let $R$ = number of rounds

The chance of the Prover getting away with cheating in a single round would be $\frac{E-1}{E}$

But every round is an independent event because the inconsitent edge is shifted, so after $R$ rounds the probability of not being caught is

$(\frac{E-1}{E})^R$

In our case $E = 15, R = 15^2$, probability of cheating $ = (\frac{14}{15})^{15^2} \approx 1.8 * 10^{-7}$



In [32]:
(14/15)**(15**2)

1.8124863313502317e-07